# **This code converts 'model_v8' into TFlite model 'model_v8.tflte'and then into .h file**<br>
Author: Jatin Kadge<br>
Roll No.: EE23D021

In [1]:
%tensorflow_version 2.1

#%tensorflow_version` only switches the major version: 1.x or 2.x.
#You set: `2.1`. This will be interpreted as: `2.x`.

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras import layers

In [3]:
# Print versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
#print('keras ' + tf.keras.version.VERSION)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.23.5
TensorFlow 2.13.0


In [4]:
tflite_model_name = 'tflite_model_v8'  # Will be given .tflite suffix
c_model_name = 'c_model_v8'       # Will be given .h suffix

In [5]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
  #Load the Saved model
  loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Wearable_devices_newdata/ML_models/model_v8.keras")

In [7]:
# View model
loaded_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 8, 6, 16)          64        
                                                                 
 conv2d_19 (Conv2D)          (None, 6, 6, 16)          784       
                                                                 
 dropout_9 (Dropout)         (None, 6, 6, 16)          0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 3, 6, 16)          0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 288)               0         
                                                                 
 dense_18 (Dense)            (None, 100)               28900     
                                                      

In [8]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

import os
basic_model_size = os.path.getsize("tflite_model_v8.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 38040 bytes


In [9]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [10]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))